In [76]:
import numpy as np

In [77]:
transition_file = 'transitionDists.json'
emission_file = 'emissionDists.json'

In [78]:
import json
with open(transition_file) as f:
    transitionDists = json.load(f)
with open(emission_file) as f:
    emissionDists = json.load(f)

In [122]:
def print_dict(d):
    for key in d:
        print(key)
        print(d.get(key))
        print("----")

In [124]:
#print_dict(transitionDists)

In [130]:
def inverse_emission(emissionDists,transitionDists):
    states = list(transitionDists.keys()) + ['<DUMMY_END_TAG>']
    ## fid all the unique observation tokens
    new_dict = {}
    for tag_key in emissionDists:
        for token_key in emissionDists.get(tag_key):
            if(new_dict.get(token_key) == None):
                new_dict[token_key] = {}
            new_dict[token_key][tag_key] = emissionDists.get(tag_key).get(token_key)
            
    for token_key in new_dict:
        emission_prob_dict = new_dict.get(token_key)
        for st in states:
            if(emission_prob_dict.get(st) == None):
                emission_prob_dict[st] = -1000
            
    return new_dict

In [131]:
def get_emission_vector(transitionDists,inverese_emission_dict,token):
    states = list(transitionDists.keys()) + ['<DUMMY_END_TAG>']
    print(states)
    emission_dict = inverese_emission_dict.get(token)
    emission_list = []
    for st in states:
        emission_list.append(emission_dict.get(st))
    return np.array(emission_list)

In [132]:
def get_transition_vector(transitionDists,tag):
    states = list(transitionDists.keys()) + ['<DUMMY_END_TAG>']
    print(states)
    trans = transitionDists.get(tag)
    transition_list = []
    for st in states:
        transition_list.append(trans.get(st))
    return np.array(transition_list)

In [133]:
inverese_emission_dict = inverse_emission(emissionDists,transitionDists)
print(len(inverese_emission_dict))
print(inverese_emission_dict.get('No'))

19673
{'DET': -6.169248699333857, 'ADV': -7.643332572111854, 'INTJ': -4.343805421853688, '<DUMMY_START_TAG>': -1000, 'PROPN': -1000, 'PUNCT': -1000, 'ADJ': -1000, 'NOUN': -1000, 'VERB': -1000, 'ADP': -1000, 'AUX': -1000, 'PRON': -1000, 'PART': -1000, 'SCONJ': -1000, 'NUM': -1000, 'CCONJ': -1000, 'X': -1000, 'SYM': -1000, '<DUMMY_END_TAG>': -1000}


In [134]:
emission_vector = get_emission_vector(transitionDists,inverese_emission_dict,'No')
print(emission_vector)

['<DUMMY_START_TAG>', 'PROPN', 'PUNCT', 'ADJ', 'NOUN', 'VERB', 'DET', 'ADP', 'AUX', 'PRON', 'PART', 'SCONJ', 'NUM', 'ADV', 'CCONJ', 'X', 'INTJ', 'SYM', '<DUMMY_END_TAG>']
[-1000.         -1000.         -1000.         -1000.
 -1000.         -1000.            -6.1692487  -1000.
 -1000.         -1000.         -1000.         -1000.
 -1000.            -7.64333257 -1000.         -1000.
    -4.34380542 -1000.         -1000.        ]


In [135]:
transition_vector = get_transition_vector(transitionDists,'ADJ')
print(transition_vector)

['<DUMMY_START_TAG>', 'PROPN', 'PUNCT', 'ADJ', 'NOUN', 'VERB', 'DET', 'ADP', 'AUX', 'PRON', 'PART', 'SCONJ', 'NUM', 'ADV', 'CCONJ', 'X', 'INTJ', 'SYM', '<DUMMY_END_TAG>']
[-11.73453985  -3.59102244  -1.9946895   -2.95636797  -0.60755631
  -4.66892648  -5.31944289  -2.53819699  -5.66843176  -4.29556826
  -3.4278208   -3.69184035  -4.90907981  -4.31336232  -3.1074166
  -6.9387493   -8.30055264  -6.71726001  -5.51793375]


In [121]:
observation = ['Professor','is','refusing','to','cooperate']

In [90]:
start_prob_dict = {}
index = 0
for key in transitionDists:
    start_prob_dict[key] = 1/len(transitionDists)
    index += 1

In [87]:
## compute P(h_t|v_1-t)
def forward_algo(observation,emissionDists,transitionDists,start_prob_dict):
    
    states = transitionDists.keys()    
    a_messages = []
    
    for i, obs in enumerate(observation):
        ## if i = 0 (first observation) then compute a1
        if(i==0):
            a_0 = 
        
                

In [88]:
forward_algo(observation,emissionDists,transitionDists,start_prob_dict)